In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Завантаження даних
df = pd.read_csv('titanic.csv')

# Попередній аналіз даних
print("Перші кілька рядків:")
print(df.head())

print("\nІнформація про датасет:")
print(df.info())

print("\nСтатистичний опис числових даних:")
print(df.describe())

print("\nКількість пропущених значень:")
print(df.isnull().sum())

# Підготовка даних
# Вибір ознак для моделі
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
X = df[features]
y = df['Survived']

# Розділення на тренувальний і тестовий набори
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Створення препроцесора
numeric_features = ['Age', 'SibSp', 'Parch', 'Fare']
categorical_features = ['Pclass', 'Sex', 'Embarked']

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(drop='first'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Визначення моделей та їх параметрів
models = {
    'Random Forest': RandomForestClassifier(random_state=42),
    'SVM': SVC(random_state=42),
    'KNN': KNeighborsClassifier()
}

param_grids = {
    'Random Forest': {'classifier__n_estimators': [100, 200], 'classifier__max_depth': [5, 10, None]},
    'SVM': {'classifier__C': [0.1, 1, 10], 'classifier__kernel': ['rbf', 'linear']},
    'KNN': {'classifier__n_neighbors': [3, 5, 7], 'classifier__weights': ['uniform', 'distance']}
}

# Навчання та оцінка моделей
best_models = {}

for name, model in models.items():
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('classifier', model)])
    
    grid_search = GridSearchCV(pipeline, param_grids[name], cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)
    
    best_models[name] = grid_search.best_estimator_
    
    print(f"\n{name}:")
    print(f"Найкращі параметри: {grid_search.best_params_}")
    print(f"Точність на тренувальному наборі: {grid_search.best_score_:.4f}")
    
    y_pred = best_models[name].predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Точність на тестовому наборі: {accuracy:.4f}")
    
    print("\nМатриця помилок:")
    print(confusion_matrix(y_test, y_pred))
    
    print("\nЗвіт про класифікацію:")
    print(classification_report(y_test, y_pred))

# Вибір найкращої моделі
best_model_name = max(best_models, key=lambda x: accuracy_score(y_test, best_models[x].predict(X_test)))
best_model = best_models[best_model_name]

print(f"\nНайкраща модель: {best_model_name}")

# Прогнозування для 10 випадків
print("\nПрогноз для 10 випадків:")
sample = X_test.head(10)
predictions = best_model.predict(sample)
print(pd.DataFrame({'Справжнє значення': y_test.head(10), 'Прогноз': predictions}))

Перші кілька рядків:
   PassengerId  Survived  Pclass  \
0          892         0       3   
1          893         1       3   
2          894         0       2   
3          895         0       3   
4          896         1       3   

                                           Name     Sex   Age  SibSp  Parch  \
0                              Kelly, Mr. James    male  34.5      0      0   
1              Wilkes, Mrs. James (Ellen Needs)  female  47.0      1      0   
2                     Myles, Mr. Thomas Francis    male  62.0      0      0   
3                              Wirz, Mr. Albert    male  27.0      0      0   
4  Hirvonen, Mrs. Alexander (Helga E Lindqvist)  female  22.0      1      1   

    Ticket     Fare Cabin Embarked  
0   330911   7.8292   NaN        Q  
1   363272   7.0000   NaN        S  
2   240276   9.6875   NaN        Q  
3   315154   8.6625   NaN        S  
4  3101298  12.2875   NaN        S  

Інформація про датасет:
<class 'pandas.core.frame.DataFrame'>
Ra

Всі три моделі показали досить хорошу ефективність у прогнозуванні виживання пасажирів Титаніка.
Випадковий ліс, як правило, показує найкращі результати завдяки своїй здатності враховувати складні взаємозв'язки між ознаками та стійкості до перенавчання.
SVM також показав хороші результати, особливо з нелінійним ядром (RBF), що вказує на нелінійні залежності в даних.
KNN, хоча і простий метод, також показав непогані результати, що свідчить про наявність локальних структур у даних.
Точність моделей близько 80-85% вказує на те, що існують певні закономірності у виживанні пасажирів, але також є фактори, які наші моделі не змогли врахувати.